# Librerias necesarias

In [37]:
import org.apache.spark.sql.functions.{conv, lower, concat, upper, lit,regexp_replace}
import org.apache.spark.sql.types.IntegerType

import org.apache.spark.sql.functions.{conv, lower, concat, upper, lit, regexp_replace}
import org.apache.spark.sql.types.IntegerType


# Funciones UDFs

In [38]:
//Parsea el nombre
def parseName = udf((nombre: String, pattern: String) => {    
    val lower = pattern.substring(2).toLowerCase()
    val index = nombre.indexOf(lower)
    val name = nombre.substring(0, index)
    val apellido = nombre.substring(index)
    val name1 = name.capitalize
    val apellido1 = apellido.capitalize
    (name1, apellido1)
})

//Crea el número de seguridad social
def nss = udf((numero: String) => {    
    "0" * (11 - numero.length()) + numero
})

parseName: org.apache.spark.sql.expressions.UserDefinedFunction
nss: org.apache.spark.sql.expressions.UserDefinedFunction


# Lectura de los datos

In [39]:
val data = spark.read.option("header","true").csv("/Users/jose/Downloads/t_mdco_tfra400_Base3M_1.0.csv")


data: org.apache.spark.sql.DataFrame = [id_cliente: string, NU_CTE: string ... 30 more fields]


# Ejecución del programa

In [40]:
val data_real = data.select(
    parseName($"desc_text", $"desc_id").as("nombre_apellido"),
    concat(col("`registro.xls`").substr(8,6), col("clave_id").substr(5,2)).as("fecha"),
    concat(upper(col("desc_id")).substr(3,2),
           upper(col("desc_id")).substr(3,1),
           upper(col("desc_id")).substr(1,1),
           concat(col("`registro.xls`").substr(10,4), col("clave_id").substr(5,2)),
           col("agrupacion_id"),
           col("CURP")     
          ).as("CURP"),
    nss(conv(col("NSS"),8,10)).as("NSS"),
    concat(upper(col("desc_id")).substr(3,2),
           upper(col("desc_id")).substr(3,1),
           upper(col("desc_id")).substr(1,1),
           concat(col("`registro.xls`").substr(10,4), col("clave_id").substr(5,2)),
           col("producto_id")
           ).as("RFC"),
    concat(lit("4152"), conv(concat(col("producto_1_number"), col("bin_2_number")), 2, 10), col("clave_id"))
    .as("PAN")
)
data_real.show

+--------------------+--------+------------------+-----------+-------------+----------------+
|     nombre_apellido|   fecha|              CURP|        NSS|          RFC|             PAN|
+--------------------+--------+------------------+-----------+-------------+----------------+
|  [Iolande, Snaddon]|19640402|SNSI640402HDFKLX84|04601204149|SNSI640402AD3|4152547077640402|
|   [Jemie, Peachman]|19820713|PEPJ820713MDFLDH08|71680198205|PEPJ820713MS9|4152547156820713|
|    [Justus, Murton]|19580905|MUMJ580905HDFUMM71|56513681524|MUMJ580905NU7|4152554629580905|
|   [Forester, Nanni]|19530805|NANF530805MDFRRL32|31258182058|NANF530805EU6|4152493161530805|
|  [Nathalia, Chiles]|19800731|CHCN800731HDFSYX28|31080683464|CHCN800731UC7|4152544053800731|
|       [Cal, Ellson]|19900630|ELEC900630HDFTPS13|84616495835|ELEC900630IL7|4152554629900630|
| [Domenic, Limpenny]|19750609|LILD750609MDFISQ18|86556998718|LILD750609XU6|4152494398750609|
|[Mallorie, Edgerley]|19681018|EDEM681018HDFMMQ33|6610812305

data_real: org.apache.spark.sql.DataFrame = [nombre_apellido: struct<_1: string, _2: string>, fecha: string ... 4 more fields]


# Correo

In [41]:
data.select(col("apellid@_text"), col("nombre_text")).
filter(!col("apellid@_text").contains("!") && col("apellid@_text").isNotNull && !col("apellid@_text").rlike(".*\\.?[0-9]+\\..*"))
.withColumn("email", concat(col("nombre_text"), lit("@"), col("apellid@_text")))
.select("email").show(100, false)


+----------------------------+
|email                       |
+----------------------------+
|isnaddon0@prlog.org         |
|jmurton2@pen.io             |
|fnanni3@ocn.ne.jp           |
|kjeavons8@bbc.co.uk         |
|roret9@mozilla.org          |
|ctrevaskusd@jalbum.net      |
|drustedi@github.io          |
|gfitzgibbonj@amazon.de      |
|adanilishink@i2i.jp         |
|lbreakeym@nsw.gov.au        |
|plowcocko@usa.gov           |
|cvoulsp@cpanel.net          |
|zbasekeq@timesonline.co.uk  |
|eallittt@google.ca          |
|kphliponz@apache.org        |
|cdugall10@cornell.edu       |
|cstackbridge11@oaic.gov.au  |
|lfronks16@sakura.ne.jp      |
|mcappleman1c@opensource.org |
|phosier1f@tuttocitta.it     |
|whannant1i@mit.edu          |
|ftoovey1k@hud.gov           |
|nwhitmore1l@miitbeian.gov.cn|
|mvolett1p@unesco.org        |
|rlebourn1w@infoseek.co.jp   |
|cstrettle1x@noaa.gov        |
|ewhiteman1y@nyu.edu         |
|toiseau28@nsw.gov.au        |
|bscarse2d@edublogs.org      |
|anys2e@